In [469]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Open Data

In [470]:
dt = pd.read_csv("data/train.csv")
dt.shape

/Users/selbo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(637774, 9)

In [471]:
def rmspe(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

## Clean Data

In [472]:
# Clear Missing Values for the target variable
dt.loc[:, "Sales"].isnull().sum()

19027

In [473]:
dt = dt.dropna( how='any', subset=['Sales'])
dt = dt.drop("Customers", axis=1)

In [474]:
sales_zeros=(dt["Sales"] == 0)
dt = dt.loc[~sales_zeros, :]

In [475]:
dt.isnull().sum()

Date                 0
Store            15580
DayOfWeek        15299
Sales                0
Open             15455
Promo            15439
StateHoliday     15560
SchoolHoliday    15547
dtype: int64

In [476]:
#Baseline
dt.loc[:, "Sales"].mean()

6836.975867325853

In [477]:
dt

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday
27,2013-01-01,353.0,2.0,3139.0,1.0,0.0,a,1.0
115,2013-01-01,335.0,2.0,2401.0,1.0,0.0,a,1.0
147,2013-01-01,512.0,2.0,2646.0,1.0,0.0,a,1.0
162,2013-01-01,494.0,2.0,3113.0,1.0,0.0,a,1.0
199,2013-01-01,530.0,2.0,2907.0,1.0,0.0,a,1.0
...,...,...,...,...,...,...,...,...
637769,2014-07-31,752.0,4.0,7259.0,1.0,1.0,0,1.0
637770,2014-07-31,753.0,NaN,9938.0,1.0,1.0,0,1.0
637771,2014-07-31,754.0,4.0,10564.0,1.0,1.0,0,NaN
637772,2014-07-31,755.0,4.0,12302.0,1.0,1.0,0,1.0


### Join with Stores

In [478]:
data_store = pd.read_csv("data/store.csv")
data_store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [479]:
data_store.shape

(1115, 10)

In [480]:
dt.shape

(512956, 8)

In [481]:
dt_full = dt.merge(data_store, left_on='Store', right_on='Store', how="left")

In [482]:
dt_full

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2013-01-01,353.0,2.0,3139.0,1.0,0.0,a,1.0,b,b,900.0,NaN,NaN,1.0,14.0,2013.0,"Feb,May,Aug,Nov"
1,2013-01-01,335.0,2.0,2401.0,1.0,0.0,a,1.0,b,a,90.0,NaN,NaN,1.0,31.0,2013.0,"Jan,Apr,Jul,Oct"
2,2013-01-01,512.0,2.0,2646.0,1.0,0.0,a,1.0,b,b,590.0,NaN,NaN,1.0,5.0,2013.0,"Mar,Jun,Sept,Dec"
3,2013-01-01,494.0,2.0,3113.0,1.0,0.0,a,1.0,b,a,1260.0,6.0,2011.0,0.0,NaN,NaN,NaN
4,2013-01-01,530.0,2.0,2907.0,1.0,0.0,a,1.0,a,c,18160.0,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512951,2014-07-31,752.0,4.0,7259.0,1.0,1.0,0,1.0,a,a,970.0,3.0,2013.0,1.0,31.0,2013.0,"Feb,May,Aug,Nov"
512952,2014-07-31,753.0,NaN,9938.0,1.0,1.0,0,1.0,d,c,540.0,11.0,2012.0,1.0,35.0,2010.0,"Mar,Jun,Sept,Dec"
512953,2014-07-31,754.0,4.0,10564.0,1.0,1.0,0,NaN,c,c,380.0,5.0,2008.0,1.0,10.0,2014.0,"Mar,Jun,Sept,Dec"
512954,2014-07-31,755.0,4.0,12302.0,1.0,1.0,0,1.0,d,c,13130.0,12.0,2003.0,0.0,NaN,NaN,NaN


## Exploratory Analysis

In [483]:
dt_full.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [484]:
dt_full.Promo.value_counts()

0.0    281859
1.0    215658
Name: Promo, dtype: int64

In [485]:
dt_full.DayOfWeek.value_counts()

6.0    85139
2.0    84995
3.0    82859
5.0    82472
1.0    80809
4.0    79312
7.0     2071
Name: DayOfWeek, dtype: int64

In [486]:
dt_full.Open.value_counts()

1.0    497501
Name: Open, dtype: int64

In [487]:
dt_full.StateHoliday.value_counts()

0      444136
0.0     52717
a         415
b          93
c          35
Name: StateHoliday, dtype: int64

In [488]:
dt_full.SchoolHoliday.value_counts()

0.0    404398
1.0     93011
Name: SchoolHoliday, dtype: int64

In [489]:
dt_full.Store.value_counts()

733.0     551
335.0     547
85.0      546
769.0     545
494.0     544
         ... 
348.0     400
972.0     383
1081.0    373
349.0     372
103.0     304
Name: Store, Length: 1115, dtype: int64

In [490]:
dt_full.StoreType.value_counts()

a    268425
d    154104
c     65902
b      8945
Name: StoreType, dtype: int64

## Feature Engineering

In [491]:
#make one-hot-encoding for the StateHoliday variable
#a = public holiday, b = Easter holiday, c = Christmas, 0 = None
dt_eng = dt_full
dt_eng.loc[: , "PublicHoliday"] = dt_full.loc[:, "StateHoliday"]=="a" 
dt_eng.loc[: , "Easter"] = dt_full.loc[:, "StateHoliday"]=="b" 
dt_eng.loc[: , "Christmas"] = dt_full.loc[:, "StateHoliday"]=="c" 
#dt_eng = dt.drop("StateHoliday", axis=1)


In [492]:
dt_full.StateHoliday.isnull().sum()

15560

In [493]:
dt_eng.PublicHoliday.isnull().sum()

0

In [494]:
#StoreType
#try either target encoding or one-hot encoding
dummies = pd.get_dummies(dt_full.loc[:, "StoreType"], prefix="storetype", prefix_sep='_')
dt_eng = pd.concat([dt_eng, dummies], axis=1)


In [495]:
#Assortment
#could be ordinal encoding
#dt_full.loc[dt_full['Assortment']=="a", "Sales"]
dt_eng.groupby("Assortment").mean()['Sales']
dummies = pd.get_dummies(dt_eng.loc[:, "Assortment"], prefix="assort", prefix_sep='_')
dt_eng = pd.concat([dt_eng, dummies], axis=1)


In [496]:
dt_eng.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'PublicHoliday',
       'Easter', 'Christmas', 'storetype_a', 'storetype_b', 'storetype_c',
       'storetype_d', 'assort_a', 'assort_b', 'assort_c'],
      dtype='object')

In [497]:
dt_eng.loc[:, ["Store", "CompetitionDistance", "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear"]]

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear
0,353.0,900.0,NaN,NaN
1,335.0,90.0,NaN,NaN
2,512.0,590.0,NaN,NaN
3,494.0,1260.0,6.0,2011.0
4,530.0,18160.0,NaN,NaN
...,...,...,...,...
512951,752.0,970.0,3.0,2013.0
512952,753.0,540.0,11.0,2012.0
512953,754.0,380.0,5.0,2008.0
512954,755.0,13130.0,12.0,2003.0


In [498]:
dt_eng.loc[dt_eng["Store"]==423, ["Store", "Sales", "Date", "CompetitionDistance", "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear"]].head(10)

,Store,Sales,Date,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear
5,423.0,9643.0,2013-01-01,1270.0,5.0,2014.0
239,423.0,9570.0,2013-01-02,1270.0,5.0,2014.0
1325,423.0,8254.0,2013-01-03,1270.0,5.0,2014.0
2402,423.0,9285.0,2013-01-04,1270.0,5.0,2014.0
3479,423.0,7095.0,2013-01-05,1270.0,5.0,2014.0
4334,423.0,10770.0,2013-01-06,1270.0,5.0,2014.0
4565,423.0,10379.0,2013-01-07,1270.0,5.0,2014.0
5642,423.0,9826.0,2013-01-08,1270.0,5.0,2014.0
6712,423.0,9259.0,2013-01-09,1270.0,5.0,2014.0
7779,423.0,8831.0,2013-01-10,1270.0,5.0,2014.0


In [499]:
#CompetitionDistance could be metric or maybe squared?


In [500]:
#Promo2 how is this working?
dt_eng.loc[:, ["Store", "Promo2", "Promo2SinceWeek", "Promo2SinceYear"]]

,Store,Promo2,Promo2SinceWeek,Promo2SinceYear
0,353.0,1.0,14.0,2013.0
1,335.0,1.0,31.0,2013.0
2,512.0,1.0,5.0,2013.0
3,494.0,0.0,NaN,NaN
4,530.0,0.0,NaN,NaN
...,...,...,...,...
512951,752.0,1.0,31.0,2013.0
512952,753.0,1.0,35.0,2010.0
512953,754.0,1.0,10.0,2014.0
512954,755.0,0.0,NaN,NaN


In [501]:
dt_eng.loc[dt_eng["Store"]==423, ["Store", "Sales", "Date", "Promo2", "Promo2SinceWeek", "Promo2SinceYear"]].head(10)

,Store,Sales,Date,Promo2,Promo2SinceWeek,Promo2SinceYear
5,423.0,9643.0,2013-01-01,0.0,NaN,NaN
239,423.0,9570.0,2013-01-02,0.0,NaN,NaN
1325,423.0,8254.0,2013-01-03,0.0,NaN,NaN
2402,423.0,9285.0,2013-01-04,0.0,NaN,NaN
3479,423.0,7095.0,2013-01-05,0.0,NaN,NaN
4334,423.0,10770.0,2013-01-06,0.0,NaN,NaN
4565,423.0,10379.0,2013-01-07,0.0,NaN,NaN
5642,423.0,9826.0,2013-01-08,0.0,NaN,NaN
6712,423.0,9259.0,2013-01-09,0.0,NaN,NaN
7779,423.0,8831.0,2013-01-10,0.0,NaN,NaN


In [502]:
#CompetitionOpenSince[Month/Year]
#should be interacted with competitionDistance


In [503]:
dt_eng.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'PublicHoliday',
       'Easter', 'Christmas', 'storetype_a', 'storetype_b', 'storetype_c',
       'storetype_d', 'assort_a', 'assort_b', 'assort_c'],
      dtype='object')

## Final Feature selection

In [504]:
features = ['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'Promo2', 'PublicHoliday', 'Easter', 'Christmas', 
                    'storetype_a', 'storetype_b', 'storetype_c', 'storetype_d', 'assort_a', 'assort_b', 'assort_c']

In [505]:
dt_eng_nomissing = dt_eng.dropna( how='any', subset=["StateHoliday", "Assortment"])

dt_eng_nomissing = dt_eng_nomissing.dropna( how='any', subset=features)


In [506]:
X = dt_eng_nomissing.loc[:, features ]
y = dt_eng_nomissing.loc[:, 'Sales']

In [507]:
X.isnull().sum()

DayOfWeek              0
Open                   0
Promo                  0
SchoolHoliday          0
CompetitionDistance    0
Promo2                 0
PublicHoliday          0
Easter                 0
Christmas              0
storetype_a            0
storetype_b            0
storetype_c            0
storetype_d            0
assort_a               0
assort_b               0
assort_c               0
dtype: int64

## Split Data in Test/Train

In [513]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42, shuffle=True, stratify=None)

## Modelling

### Baseline

In [534]:
def rmspe(preds, actuals):
    #preds = preds.reshape(-1)
    #actuals = actuals.reshape(-1)
    #assert preds.shape == actuals.shape
    return np.sqrt(np.mean(np.square((actual-pred) / actual))) * 100

In [535]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.predict(X_test)
pred = lr.predict(X_test)
actual = y_test
rmspe(lr.predict(X_test), y_test)

50.27323475335008